In [ ]:
# save new density

# repeat from step 2
# propagate density with velocity verlet

### generate start density
- run PROFESS for one step
- print density -> initial guess
- see /home/misa/git_repositories/PROFESS/test/single_iteration

### call profess-1
- copy intial density
- call profess-1
- do it from python

### propagate density
- load density and gradient into python
- reshape
- propagate (with velocity verlet)

In [1]:
import numpy as np

In [2]:
def load_density(dens_file):
    with open(dens_file, 'r') as f:
        file = f.readlines()
        density_unparsed = file[0][108:]
        density = density_unparsed.split()
        density = np.array([float(i) for i in density])
        density = density.reshape((16,16,16))
        return(density)

In [3]:
density = load_density('/home/misa/software/PROFESS-1/calculate_gradient/density_0')
density2 = load_density('/home/misa/software/PROFESS-1/calculate_gradient/new_density_test')
np.array_equal(density, density2)

True

In [4]:
# load gradient
filep = '/home/misa/software/PROFESS-1/calculate_gradient/dEdX'
dEdX = []
with open(filep, 'r') as f:
    for i, line in enumerate(f):
        if i != 0:
            dEdX.append(float(line.strip('\n')))
dEdX = np.array(dEdX)
dEdX = dEdX.reshape((16, 16, 16))

In [5]:
# what are the units of density and dE/dX?
import ase.units as au
V = (3.9691/au.Bohr)**3
dV = V/16**3
density_vox = density*dV
dEdX_vox = dEdX*dV

In [6]:
print((density*dV).sum())
print("Density must be converted to Voxel/Bohr^3")

11.999999866902229
Density must be converted to Voxel/Bohr^3


#### What are appropiate values for mu and dt?
- $\mu = 1$ and $dt = 0.1$ (both in a.u.) (10.1103/physrevlett.55.2471)

In [ ]:
mu = 1
dt = 0.1

In [7]:
# propagate density  first step
a = -dEdX_vox/mu
x_1 = a*dt**2 + np.sqrt(density_vox)

In [ ]:
# propagate density velocity verlet
def vv_step(X, X_m, dt, mu, dEdX):
    X_p = 2*X - X_m - (dt**2/mu)*dEdX
    return(X_p)

# load density from previous step
dens_prev = load_density('/home/misa/software/PROFESS-1/calculate_gradient/density_0')
dens_prev = dens_prev*dV
X_minus = np.sqrt(dens_prev)
X = np.sqrt(density_vox)

In [ ]:
x_1 = vv_step(X, X_minus, dt, mu, dEdX_vox)

### enforce constraint

In [8]:
Ne = 12.0
N_p = (x_1*np.sqrt(density_vox)).sum()
N_pp = (x_1*x_1).sum()
tau = dt**2/mu

In [9]:
p2 = N_p/(tau*Ne)
q = (N_pp - Ne)/(tau**2*Ne)
lambda_2 = -p2 + np.sqrt(p2**2 - q)

In [10]:
x_1_constr = x_1 + dt**2/mu*lambda_2*x_1
density_1 = np.power(x_1_constr,2)

In [11]:
density_1.sum()

11.999908874690611

### save new density

In [12]:
start_str = '  x-dimension:          16   y-dimension:          16   z-dimension:          16   # of spins:            1 '
density_1_fl = density_1.flatten()/dV
for d in density_1_fl:
    start_str += "{:.20E} ".format(d)
    
new_dens = '/home/misa/software/PROFESS-1/calculate_gradient/new_density'
with open(new_dens, 'w') as f:
    f.write(start_str)

In [ ]:
# start_str = '  x-dimension:          16   y-dimension:          16   z-dimension:          16   # of spins:            1 '
# density_1_fl = density.flatten()
# length = len(start_str)
# for d in density_1_fl:
#     start_str += "{:.20E} ".format(d)
#     length += len(str(d)) + 1
    
# new_dens = '/home/misa/software/PROFESS-1/calculate_gradient/new_density_test'
# with open(new_dens, 'w') as f:
#     f.write(start_str)
#     #f.write('\n')

In [ ]:
#len('0.62344209078022891357E-02')
#len('3.04456931547079749645E-02')